In [1]:
# coding=utf-8
# Copyright 2023-present the HuggingFace Inc. team.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForVision2Seq, AutoProcessor
from PIL import Image

from tqdm import tqdm
import csv

In [2]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
dataset = load_dataset("csv", data_files={"train" : "./ambiguous_questions_train.csv", "test" : "./ambiguous_questions_test.csv"})

Using custom data configuration default-b4c98d92954de12c
Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-b4c98d92954de12c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model_name_or_path = 'Salesforce/blip2-flan-t5-xxl'
cache_dir = "./" + model_name_or_path.split('/')[-1]

# We load our model and processor using `transformers`
processor = AutoProcessor.from_pretrained(model_name_or_path,cache_dir=cache_dir)
model = AutoModelForVision2Seq.from_pretrained(model_name_or_path,cache_dir=cache_dir, device_map="sequential",torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)
model.eval()


Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

In [10]:
with open("./ambiguous_questions_train.csv", 'r') as f:
    reader = csv.reader(f)
    train_lines = [line for line in reader]


with open("./ambiguous_questions_test.csv", 'r') as f:
    reader = csv.reader(f)
    test_lines = [line for line in reader]
    
    
  

promt = f"Instructions : Classify the following into ambigous and definite question. An ambiguous question is unanswerable question considering image. "

few_shot_image_ids = [train_lines[idx][5] for idx in range(1,6)]
images = [Image.open("./images/" + str(id) + '.jpg').convert('RGB') for id in few_shot_image_ids]

predictions = []
for idx, line in tqdm(enumerate(test_lines)):
    if idx == 0:
        continue
    question = line[1]
    image_id = line[5]
    image = Image.open("./images/" + str(image_id) + '.jpg').convert('RGB')
    input_promt = promt + "qustion : {} label : ".format(question)
   

    inputs = processor(image, input_promt, return_tensors="pt").to(device, torch.float16)
    out = model.generate(**inputs)
    prediction = processor.decode(out[0], skip_special_tokens=True)
    print(prediction)
    predictions.append(prediction)
    
with open ("./test_fewshot.csv", 'w') as f:
        
    writer = csv.writer(f)
    for idx, line in enumerate(test_lines):
        if idx == 0:
            writer.writerow(line)
        else:
            line.append(predictions[idx-1])
            writer.writerow(line)
                
            
                

0it [00:00, ?it/s]/opt/conda/envs/torch2.0/lib/python3.9/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
3it [00:00,  7.26it/s]

definite
definite


5it [00:00,  5.84it/s]

definite
definite


7it [00:01,  5.59it/s]

...

definite
definite


99it [00:18,  5.45it/s]

definite
definite


101it [00:18,  5.34it/s]

definite
definite
